Лабораторная работа № 5: **Деревья решений**<br>
Выполнил: **Стадников Александр, гр.6135**

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style("darkgrid")

import warnings
import sys  
if not sys.warnoptions:
    warnings.simplefilter("ignore")

# Загрузка датасета

**Исходные данные:** https://www.kaggle.com/uciml/student-alcohol-consumption?select=student-mat.csv 

**Набор данных:** Потребление алкоголя студентами

Данные были получены в результате опроса студентов, обучающихся на курсах математики и португальского языка в средней школе. Он содержит много интересной социальной, гендерной и учебной информации о студентах.

1. **school** - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. **sex** - student's sex (binary: 'F' - female or 'M' - male)
3. **age** - student's age (numeric: from 15 to 22)
4. **address** - student's home address type (binary: 'U' - urban or 'R' - rural)
5. **famsize** - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. **Pstatus** - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. **Medu** - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
8. **Fedu** - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 – 5th to 9th grade, 3 – secondary education or 4 – higher education)
9. **Mjob** - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. **Fjob** - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. **reason** - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. **guardian** - student's guardian (nominal: 'mother', 'father' or 'other')
13. **traveltime** - home to school travel time (numeric: 1 - 1 hour)
14. **studytime** - weekly study time (numeric: 1 - 10 hours)
15. **failures** - number of past class failures (numeric: n if 1<=n<3, else 4)
16. **schoolsup** - extra educational support (binary: yes or no)
17. **famsup** - family educational support (binary: yes or no)
18. **paid** - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. **activities** - extra-curricular activities (binary: yes or no)
20. **nursery** - attended nursery school (binary: yes or no)
21. **higher** - wants to take higher education (binary: yes or no)
22. **internet** - Internet access at home (binary: yes or no)
23. **romantic** - with a romantic relationship (binary: yes or no)
24. **famrel** - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. **freetime** - free time after school (numeric: from 1 - very low to 5 - very high)
26. **goout** - going out with friends (numeric: from 1 - very low to 5 - very high)
27. **Dalc** - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. **Walc** - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. **health** - current health status (numeric: from 1 - very bad to 5 - very good)
30. **absences** - number of school absences (numeric: from 0 to 93)

These grades are related with the course subject, Math or Portuguese:

1. **G1** - first period grade (numeric: from 0 to 20)
2. **G2** - second period grade (numeric: from 0 to 20)
3. **G3** - final grade (numeric: from 0 to 20, output target)

In [2]:
s_data = pd.read_csv('student-mat.csv')
s_data.head(3)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10


In [3]:
# удалим числовые переменные
# Walc - целевая
df = s_data.drop(['age','traveltime',
                  'studytime','absences','G1','G2','G3'], axis=1)

# 1. Преобразование категориальных признаков в бинарные
- с помощью pd.factorize / pd.get_dummies

In [4]:
df.school = pd.Series(map(lambda x: dict(GP=1, MS=0)[x], df.school.tolist())) # 1 = GP, 0 = MS
df.sex = pd.Series(map(lambda x: dict(M=1, F=0)[x], df.sex.tolist())) # 1 = Male, 0 = Female
df.address = pd.Series(map(lambda x: dict(U=1, R=0)[x], df.address.tolist())) # 1 = Urban, 0 = Rural
df.famsize = pd.Series(map(lambda x: dict(GT3=1, LE3=0)[x], df.famsize.tolist()))
df.Pstatus = pd.Series(map(lambda x: dict(A=1, T=0)[x], df.Pstatus.tolist()))
df.schoolsup = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.schoolsup.tolist()))
df.famsup = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.famsup.tolist()))
df.paid = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.paid.tolist()))
df.activities = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.activities.tolist()))
df.nursery = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.nursery.tolist()))
df.higher = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.higher.tolist()))
df.internet = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.internet.tolist()))
df.romantic = pd.Series(map(lambda x: dict(yes=1, no=0)[x], df.romantic.tolist()))

In [5]:
encoding_Medu = pd.get_dummies(s_data.Medu)
encoding_Fedu = pd.get_dummies(s_data.Fedu)
encoding_Mjob = pd.get_dummies(s_data.Mjob)
encoding_Fjob = pd.get_dummies(s_data.Fjob)
encoding_reason = pd.get_dummies(s_data.reason)
encoding_guardian = pd.get_dummies(s_data.guardian)
encoding_freetime = pd.get_dummies(s_data.freetime)
encoding_goout = pd.get_dummies(s_data.goout)
encoding_health = pd.get_dummies(s_data.health)
encoding_Dalc = pd.get_dummies(s_data.Dalc)
# целевая переменная
df['encoding_Walc'], _ = pd.factorize(s_data.Walc)
encoding_failures = pd.get_dummies(s_data.failures)
encoding_famrel = pd.get_dummies(s_data.famrel)

In [6]:
df = df.drop(['Medu','Fedu','Mjob','Fjob','reason',
              'guardian','failures','famrel','freetime',
              'goout', 'Dalc', 'Walc', 'health'], axis=1)

In [7]:
df = pd.concat([df,encoding_Medu,
                encoding_Fedu,encoding_Mjob,encoding_Fjob,
                encoding_reason,encoding_guardian,
                encoding_failures,encoding_famrel,
                encoding_freetime,encoding_goout,
                encoding_Dalc,encoding_health
                       ], axis=1)
df.head(3)

,school,sex,address,famsize,Pstatus,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,encoding_Walc,0,1,2,3,4,0,1,2,3,4,at_home,health,other,services,teacher,at_home,health,other,services,teacher,course,home,other,reputation,father,mother,other,0,1,2,3,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5,1,2,3,4,5
0,1,0,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1,1,0,1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
2,1,0,1,0,0,1,0,1,0,1,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0


**Переименуем колонки, переместим целевую переменную на первое место**

In [8]:
df.columns = ['school','sex','address','famsize','Pstatus',
              'schoolsup','famsup','paid','activities',
              'nursery','higher','internet','romantic',
              'encoding_Walc',
              'Medu_0','Medu_1','Medu_2','Medu_3','Medu_4',
              'Fedu_0','Fedu_1','Fedu_2','Fedu_3','Fedu_4',
              'Mjob_home','Mjob_health','Mjob_other','Mjob_services','Mjob_teacher',
              'Fjob_home','Fjob_health','Fjob_other','Fjob_services','Fjob_teacher',
              'reason_course','reason_home','reason_other','reason_reputation',
              'g_father','g_mother','g_other',
              'fail_0','fail_1','fail_2','fail_3',
              'famrel_vbad','famrel_bad','famrel_normal','famrel_good','famrel_vgood',
              'freeT_vlow','freeT_low','freeT_normal','freeT_high','freeT_vhigh',
              'goout_vlow','goout_low','goout_normal','goout_high','goout_vhigh',
              'Dalc_vlow','Dalc_low','Dalc_normal','Dalc_high','Dalc_vhigh',
              'health_vbad','health_bad','health_normal','health_good','health_vgood'
                     ]

In [9]:
cols = df.columns.tolist()
cols = ['encoding_Walc','school', 'sex', 'address', 'famsize',
        'Pstatus','schoolsup', 'famsup', 'paid', 'activities',
        'nursery', 'higher', 'internet', 'romantic',
        'Medu_0', 'Medu_1', 'Medu_2', 'Medu_3', 'Medu_4', 'Fedu_0',
        'Fedu_1', 'Fedu_2', 'Fedu_3', 'Fedu_4', 'Mjob_home',
        'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
        'Fjob_home', 'Fjob_health', 'Fjob_other', 'Fjob_services',
        'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other',
        'reason_reputation', 'g_father', 'g_mother', 'g_other', 'fail_0',
        'fail_1', 'fail_2', 'fail_3', 'famrel_vbad', 'famrel_bad',
        'famrel_normal', 'famrel_good', 'famrel_vgood', 'freeT_vlow',
        'freeT_low', 'freeT_normal', 'freeT_high', 'freeT_vhigh',
        'goout_vlow', 'goout_low', 'goout_normal', 'goout_high',
        'goout_vhigh', 'Dalc_vlow', 'Dalc_low', 'Dalc_normal',
        'Dalc_high', 'Dalc_vhigh', 'health_vbad', 'health_bad',
        'health_normal', 'health_good', 'health_vgood']
df = df[cols]
df.head(3)

,encoding_Walc,school,sex,address,famsize,Pstatus,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,Medu_0,Medu_1,Medu_2,Medu_3,Medu_4,Fedu_0,Fedu_1,Fedu_2,Fedu_3,Fedu_4,Mjob_home,Mjob_health,Mjob_other,Mjob_services,Mjob_teacher,Fjob_home,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,g_father,g_mother,g_other,fail_0,fail_1,fail_2,fail_3,famrel_vbad,famrel_bad,famrel_normal,famrel_good,famrel_vgood,freeT_vlow,freeT_low,freeT_normal,freeT_high,freeT_vhigh,goout_vlow,goout_low,goout_normal,goout_high,goout_vhigh,Dalc_vlow,Dalc_low,Dalc_normal,Dalc_high,Dalc_vhigh,health_vbad,health_bad,health_normal,health_good,health_vgood
0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0
1,0,1,0,1,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0
2,1,1,0,1,0,0,1,0,1,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0


## 1.1. С помощью sklearn.preprocessing.StandardScaler нормализовать значения признаков и построить для нормализовапнного датасета KNeighborsClassifier

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [11]:
features_values = df.iloc[:, 1:]
target_value = df['encoding_Walc']
X_train, X_test, y_train, y_test = train_test_split(features_values, target_value, test_size=0.3, random_state=42)

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled =  scaler.transform(X_test)

In [13]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train_scaled, y_train)
predictions = knn_classifier.predict(X_test_scaled)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

score = cross_val_score(knn_classifier, X_train_scaled, y_train, cv=5, scoring='accuracy')
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.4093506493506494
Точность на тестовой выборке:  0.3445378151260504


## 1.2. То же самое и использованием sklearn.pipeline.Pipeline

In [15]:
from sklearn.pipeline import Pipeline

In [16]:
pipe_knn = Pipeline([('scaler', StandardScaler()), 
                     ('kNN', KNeighborsClassifier())]
                    )
pipe_knn.fit(X_train, y_train)
predictions = pipe_knn.predict(X_test)

score = cross_val_score(pipe_knn, X_train, y_train, cv=5, scoring='accuracy')
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.42753246753246754
Точность на тестовой выборке:  0.3445378151260504


## 1.3. GridSearchCV
С помощью GridSearchCV произвести подбор гиперпараметра KNeighborsClassifier (n_neighbors) и выведите значение гиперпараметра и метрику для наилучшей модели

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
grid_params = {
    'n_neighbors': [*range(1, 11, 1)]
}

search = GridSearchCV(KNeighborsClassifier(),
                      param_grid=grid_params, 
                      scoring=['accuracy'], 
                      n_jobs=-1, 
                      cv=5,
                      verbose=3,
                      refit='accuracy',
                      )
searching_results = search.fit(X_train, y_train)
pridictions = search.predict(X_test)
print(f'Лучшие параметры {search.best_params_}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    6.3s


Лучшие параметры {'n_neighbors': 10}


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.6s finished


In [19]:
knn_classifier = KNeighborsClassifier(n_neighbors=6)
knn_classifier.fit(X_train_scaled, y_train)
predictions = knn_classifier.predict(X_test_scaled)

score = cross_val_score(pipe_knn, X_train, y_train, cv=5, scoring='accuracy')
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.42753246753246754
Точность на тестовой выборке:  0.3865546218487395


# 2.1. Продемонстрировать работу DecisionTreeClassifier, посчитать метрики качества на отложенной выборке

In [20]:
from sklearn.tree import DecisionTreeClassifier

In [21]:
dtree = DecisionTreeClassifier(random_state=39)
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)

score = cross_val_score(dtree, X_train, y_train, cv=5, scoring='accuracy')
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.34454545454545454
Точность на тестовой выборке:  0.3697478991596639


## 2.2.  GridSearchCV
С помощью GridSearchCV произвести подбор гиперпараметров для поиск наилучшего дерева (перебрать max_depth, max_features и может быть что-то ещё)

In [22]:
# параметры для дерева решений
params = dict(max_depth = [*range(1, 10, 1)], 
              max_features = [*range(1, 7, 1)],
              min_samples_split = [*range(2, 12, 1)],
              min_samples_leaf = [*range(1, 10, 1)]
              )

In [23]:
search = GridSearchCV(DecisionTreeClassifier(random_state=42),
                      param_grid=params, 
                      scoring=['accuracy'], 
                      n_jobs=-1, 
                      cv=5,
                      verbose=3,
                      refit='accuracy',
                      )
searching_results = search.fit(X_train, y_train)
pridictions = search.predict(X_test)
print(f'Лучшие параметры {search.best_params_}')

Fitting 5 folds for each of 4860 candidates, totalling 24300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1904 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 3696 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 6000 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 8816 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 12144 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-1)]: Done 15984 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 20336 tasks      | elapsed:  1.3min


Лучшие параметры {'max_depth': 6, 'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 2}


[Parallel(n_jobs=-1)]: Done 24300 out of 24300 | elapsed:  1.5min finished


In [24]:
dtree = DecisionTreeClassifier(random_state=47,
                               max_depth=6,
                               max_features=4,
                               min_samples_split=2,
                               min_samples_leaf=1,
                               )
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)

score = cross_val_score(dtree, X_train, y_train, cv=5, scoring='accuracy')
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.4057142857142857
Точность на тестовой выборке:  0.3949579831932773


## 2.3. feature_importances

Для полученного наилучшего дерева построить график feature_importances и дополнительно напечатать 3 наиболее важных признака, отсортиров их по убыванию по важности

In [25]:
print("Важность признаков:")
print(dtree.feature_importances_ )

Важность признаков:
[0.         0.         0.01991619 0.03365073 0.         0.07059181
 0.         0.         0.01327746 0.         0.04216933 0.01991619
 0.0088114  0.         0.02489523 0.07626732 0.02389942 0.
 0.         0.         0.00663873 0.         0.         0.
 0.         0.02589104 0.         0.         0.         0.
 0.00663873 0.01991619 0.         0.01194971 0.         0.
 0.04779884 0.05133392 0.         0.01194971 0.01973775 0.
 0.03726767 0.         0.         0.         0.         0.
 0.         0.02554503 0.03245601 0.02309887 0.02731363 0.
 0.02025709 0.00929422 0.         0.01822633 0.11838008 0.1091583
 0.02021795 0.         0.         0.         0.         0.
 0.         0.02353515 0.        ]


In [26]:
feature_names = features_values.columns

In [27]:
for name, score in zip(df[feature_names], dtree.feature_importances_):
    print(name, ':', score)

school : 0.0
sex : 0.0
address : 0.019916185366706463
famsize : 0.03365072989665021
Pstatus : 0.0
schoolsup : 0.07059181257754847
famsup : 0.0
paid : 0.0
activities : 0.013277456911137643
nursery : 0.0
higher : 0.042169326154006134
internet : 0.019916185366706463
romantic : 0.008811403222845862
Medu_0 : 0.0
Medu_1 : 0.024895231708383078
Medu_2 : 0.07626731514524135
Medu_3 : 0.02389942244004774
Medu_4 : 0.0
Fedu_0 : 0.0
Fedu_1 : 0.0
Fedu_2 : 0.006638728455568819
Fedu_3 : 0.0
Fedu_4 : 0.0
Mjob_home : 0.0
Mjob_health : 0.0
Mjob_other : 0.025891040976718415
Mjob_services : 0.0
Mjob_teacher : 0.0
Fjob_home : 0.0
Fjob_health : 0.0
Fjob_other : 0.006638728455568819
Fjob_services : 0.019916185366706463
Fjob_teacher : 0.0
reason_course : 0.01194971122002387
reason_home : 0.0
reason_other : 0.0
reason_reputation : 0.04779884488009552
g_father : 0.05133392129299084
g_mother : 0.0
g_other : 0.01194971122002387
fail_0 : 0.019737745015592834
fail_1 : 0.0
fail_2 : 0.037267665381347057
fail_3 : 0.0
fa

------
> **Наиболее важные признаки:** Dalc_low, Dalc_vhigh, freeT_low (Потребление алкоголя в рабочие дни, кол-во свободного времени)

# 3.1. RandomForestClassifier
- Продемонстрировать работу случайного леса RandomForestClassifier

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=8,
                            n_jobs=-1, 
                            random_state=42,
                            )
rf = rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
score = cross_val_score(rf, X_train, y_train, cv=5)
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.5001298701298701
Точность на тестовой выборке:  0.4957983193277311


## 3.2. GridSearchCV
С помощью GridSearchCV произвести подбор гиперпараметров для поиск наилучшего леса, посмотреть насколько улучшились метрики

In [29]:
n_estimators = [*range(100, 200, 10)]
max_features = ['auto', 'sqrt']
max_depth = [*range(1, 11, 1)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [30]:
search = GridSearchCV(RandomForestClassifier(random_state=42),
                      param_grid=random_grid, 
                      scoring=['accuracy'], 
                      n_jobs=-1, 
                      cv=5,
                      verbose=3,
                      refit='accuracy',
                      )
searching_results = search.fit(X_train, y_train)
pridictions = search.predict(X_test)
print(f'Лучшие параметры {search.best_params_}')

Fitting 5 folds for each of 3960 candidates, totalling 19800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 5392 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 6256 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | e

Лучшие параметры {'bootstrap': True, 'max_depth': 7, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 110}


In [32]:
rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=7,
                            max_features='auto',
                            bootstrap=True,
                            n_jobs=-1, 
                            random_state=42,
                            min_samples_leaf=2,
                            min_samples_split = 10
                            )
rf = rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
score = cross_val_score(rf, X_train, y_train, cv=5)
print('Средняя точность: ', np.mean(score))
print('Точность на тестовой выборке: ', accuracy_score(y_test, predictions))

Средняя точность:  0.5218181818181817
Точность на тестовой выборке:  0.5294117647058824


**Лучший результат дал алгоритм "Random Forest" с настройкой гиперпарамететров.**

>Средняя точность:  `0.5218181818181817`<br>
Точность на тестовой выборке:  `0.5294117647058824`